In [1]:
## This file implements neural networks and random forest on p003ppresabs_qual.
## Due to the imbalanced data and the limited cases for class 2, we implement the over-sampling method method.
## For fully-connected neural networks, the accuracy is 97.06% for over-sampling data and 93.63% after regularization.
## For random forest, the accuracy is 98.04% for over-sampling data.
## For random forest with cross-validation, the mean accuracy is 98.94%.

In [27]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [28]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_qual/p003ppresabs_qual.csv')
df.shape

(255, 866)

In [29]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [30]:
df['pheno']

0      0
1      0
2      1
3      0
4      0
5      0
6      0
7      0
8      0
9      0
10     0
11     0
12     0
13     0
14     0
15     0
16     0
17     0
18     0
19     0
20     0
21     0
22     0
23     0
24     1
25     0
26     1
27     0
28     0
29     0
      ..
225    0
226    0
227    0
228    1
229    1
230    0
231    0
232    0
233    0
234    0
235    0
236    0
237    0
238    0
239    0
240    0
241    0
242    0
243    0
244    0
245    0
246    0
247    0
248    0
249    0
250    0
251    0
252    0
253    0
254    0
Name: pheno, Length: 255, dtype: int64

In [31]:
df['pheno'].value_counts()

0    226
1     26
2      3
Name: pheno, dtype: int64

In [32]:
df.head()

,id,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATG,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGA,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAA,TTTTTTCTTTTCATAACTGTGTTGGAAATGAATTAAATTAACAGCTCTTTGTGCTTTACGGTGTGTTGC,TTTTTTCAGCATTGTCTACATTACTTAACATTCGTGTTTGTAAGTAATATTGACCGCCAATATTTAGACACTTTATAAGTATGCCATTCATCATTTTTAA,TTTTTTATCTCACCAATTTTGTAATACATCGTTCTCGTCCTCCTTGTCTTCTTCGTCCTCCTCGTTATCTTCTTCGTTTTGTAATTCATAAATTTTGTTT,TTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAAA,TTTTTTAGGTACC,TTTTTGCATTCA,...,group_8071,group_8208,group_8664,group_8665,group_8666,group_8667,group_8836,group_8913,group_9508,pheno
0,107,1,1,1,1,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,109,1,1,1,1,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,115,1,1,1,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,1
3,120335,1,1,1,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,120337,1,1,1,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [33]:
df_clean = df.drop(columns=['id'])

In [34]:
df_clean.shape

(255, 865)

In [35]:
df_clean.head()

,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATG,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGA,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAA,TTTTTTCTTTTCATAACTGTGTTGGAAATGAATTAAATTAACAGCTCTTTGTGCTTTACGGTGTGTTGC,TTTTTTCAGCATTGTCTACATTACTTAACATTCGTGTTTGTAAGTAATATTGACCGCCAATATTTAGACACTTTATAAGTATGCCATTCATCATTTTTAA,TTTTTTATCTCACCAATTTTGTAATACATCGTTCTCGTCCTCCTTGTCTTCTTCGTCCTCCTCGTTATCTTCTTCGTTTTGTAATTCATAAATTTTGTTT,TTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAAA,TTTTTTAGGTACC,TTTTTGCATTCA,TTTTTCTTATTTTA,...,group_8071,group_8208,group_8664,group_8665,group_8666,group_8667,group_8836,group_8913,group_9508,pheno
0,1,1,1,1,1,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,1,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
3,1,1,1,0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [36]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 864) (255,)


In [37]:
# over-sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 226), (1, 226), (2, 226)]


In [38]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [39]:
############# Fully-Connected Neural Network ################

In [40]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [41]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [42]:
model1_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [43]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 474 samples, validate on 204 samples
Epoch 1/100
474/474 [==============================] - 0s 253us/step - loss: 0.8455 - accuracy: 0.6097 - val_loss: 0.5854 - val_accuracy: 0.7892
Epoch 2/100
474/474 [==============================] - 0s 79us/step - loss: 0.5002 - accuracy: 0.8122 - val_loss: 0.4190 - val_accuracy: 0.8431
Epoch 3/100
474/474 [==============================] - 0s 81us/step - loss: 0.3799 - accuracy: 0.8544 - val_loss: 0.3511 - val_accuracy: 0.8873
Epoch 4/100
474/474 [==============================] - 0s 85us/step - loss: 0.3049 - accuracy: 0.9030 - val_loss: 0.3127 - val_accuracy: 0.8922
Epoch 5/100
474/474 [==============================] - 0s 80us/step - loss: 0.2628 - accuracy: 0.9219 - val_loss: 0.2713 - val_accuracy: 0.8971
Epoch 6/100
474/474 [==============================] - 0s 81us/step - loss: 0.2497 - accuracy: 0.9093 - val_loss: 0.2574 - val_accuracy: 0.9265
Epoch 7/100
474/474 [==============================] - 0s 93us/step - loss: 0.2133 - accu

474/474 [==============================] - 0s 116us/step - loss: 0.0074 - accuracy: 1.0000 - val_loss: 0.1352 - val_accuracy: 0.9657
Epoch 58/100
474/474 [==============================] - 0s 93us/step - loss: 0.0071 - accuracy: 1.0000 - val_loss: 0.1204 - val_accuracy: 0.9755
Epoch 59/100
474/474 [==============================] - 0s 70us/step - loss: 0.0068 - accuracy: 1.0000 - val_loss: 0.1151 - val_accuracy: 0.9755
Epoch 60/100
474/474 [==============================] - 0s 66us/step - loss: 0.0067 - accuracy: 1.0000 - val_loss: 0.1220 - val_accuracy: 0.9755
Epoch 61/100
474/474 [==============================] - 0s 67us/step - loss: 0.0068 - accuracy: 1.0000 - val_loss: 0.1111 - val_accuracy: 0.9755
Epoch 62/100
474/474 [==============================] - 0s 66us/step - loss: 0.0064 - accuracy: 1.0000 - val_loss: 0.1126 - val_accuracy: 0.9755
Epoch 63/100
474/474 [==============================] - 0s 65us/step - loss: 0.0063 - accuracy: 1.0000 - val_loss: 0.1282 - val_accuracy: 0.97

In [44]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

204/204 [==============================] - 0s 55us/step
over-sampling test accuracy: 97.06%


In [45]:
#### add regularizor and dropout
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [46]:
model1_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [47]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 474 samples, validate on 204 samples
Epoch 1/100
474/474 [==============================] - 0s 386us/step - loss: 4.3836 - accuracy: 0.5021 - val_loss: 0.8018 - val_accuracy: 0.7108
Epoch 2/100
474/474 [==============================] - 0s 87us/step - loss: 3.4635 - accuracy: 0.6329 - val_loss: 0.7437 - val_accuracy: 0.8431
Epoch 3/100
474/474 [==============================] - 0s 89us/step - loss: 3.3835 - accuracy: 0.6688 - val_loss: 0.7138 - val_accuracy: 0.8480
Epoch 4/100
474/474 [==============================] - 0s 86us/step - loss: 2.8955 - accuracy: 0.7046 - val_loss: 0.7403 - val_accuracy: 0.8333
Epoch 5/100
474/474 [==============================] - 0s 84us/step - loss: 3.1313 - accuracy: 0.6730 - val_loss: 0.8310 - val_accuracy: 0.8529
Epoch 6/100
474/474 [==============================] - 0s 86us/step - loss: 2.8966 - accuracy: 0.6814 - val_loss: 0.7809 - val_accuracy: 0.8627
Epoch 7/100
474/474 [==============================] - 0s 85us/step - loss: 2.8736 - accu

474/474 [==============================] - 0s 96us/step - loss: 1.1349 - accuracy: 0.8017 - val_loss: 0.8038 - val_accuracy: 0.9314
Epoch 58/100
474/474 [==============================] - 0s 88us/step - loss: 1.0517 - accuracy: 0.8312 - val_loss: 0.7175 - val_accuracy: 0.9363
Epoch 59/100
474/474 [==============================] - 0s 81us/step - loss: 1.1052 - accuracy: 0.8101 - val_loss: 0.8310 - val_accuracy: 0.9363
Epoch 60/100
474/474 [==============================] - 0s 74us/step - loss: 1.1269 - accuracy: 0.8143 - val_loss: 0.8590 - val_accuracy: 0.9314
Epoch 61/100
474/474 [==============================] - 0s 72us/step - loss: 1.1002 - accuracy: 0.8017 - val_loss: 0.8594 - val_accuracy: 0.9265
Epoch 62/100
474/474 [==============================] - 0s 78us/step - loss: 1.1958 - accuracy: 0.7911 - val_loss: 0.8329 - val_accuracy: 0.9314
Epoch 63/100
474/474 [==============================] - 0s 76us/step - loss: 1.3545 - accuracy: 0.7827 - val_loss: 0.7573 - val_accuracy: 0.941

In [48]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

204/204 [==============================] - 0s 85us/step
over-sampling test accuracy: 93.63%


In [49]:
############## Random Forest ##############

In [50]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [51]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 98.04%


In [52]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rfcv_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.98958333 1.         0.97916667 1.         0.97849462]
0.9894489247311828
